# 蔬菜价格预测模型

2018年4月25日

在这个项目中，我们将使用成都市蔬菜价格数据建立一个有关最高价和最低价的预测模型。探索多个常用的监督学习算法并找出其中最优的方案。

## 1. 数据准备

### 1.1 导入数据

首先从CSV文件中导入数据，计算每一条数据前3天和前9天价格平均值，然后将数据拆分为特征和目标两个部分。

In [198]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [199]:
#读取成都的蔬菜价格数据
#data = pd.read_csv('./vegetable_prices.csv')
#读取重庆的蔬菜价格数据
data = pd.read_csv('./vegetable_prices_Chongqing.csv')
# calculating average prices 3 or 9 days before
data['h_3'] = data['h_price'].shift(1).rolling(window=3).mean()
data['h_9'] = data['h_price'].shift(1).rolling(window=9).mean()
data['l_3'] = data['l_price'].shift(1).rolling(window=3).mean()
data['l_9'] = data['l_price'].shift(1).rolling(window=9).mean()
data = data.dropna()

prices = data[['h_price', 'l_price']]
features = data.drop(['l_price', 'h_price', 'v_price'], axis=1)
print('chengdu vegetable dataset has {} data points with {} variables each'.format(*data.shape))

chengdu vegetable dataset has 5225 data points with 22 variables each


### 1.2 分析数据

数据包含如下22个变量：

In [200]:
data.head()

,v_name,v_price,h_price,l_price,v_market,area,source,updateTime,insertTime,lWendu,...,fengli,fengxiang,aqi,aqiLevel,aqiInfo,cpi,h_3,h_9,l_3,l_9
9,芹菜,2.5,2.6,2.4,重庆双福国际农贸城,cq,vegnet.com.cn,2016-11-10,2018-04-24,9,...,微风,无持续风向,41,1,优,102.0,8.933333,5.577778,7.933333,4.800000
10,莴笋,4.0,5.0,3.0,重庆双福国际农贸城,cq,vegnet.com.cn,2016-11-10,2018-04-24,9,...,微风,无持续风向,41,1,优,102.0,9.000000,5.355556,7.933333,4.622222
11,莲藕,3.9,5.0,2.8,重庆双福国际农贸城,cq,vegnet.com.cn,2016-11-10,2018-04-24,9,...,微风,无持续风向,41,1,优,102.0,7.733333,5.688889,6.133333,4.777778
12,茭白,2.2,2.4,2.0,重庆双福国际农贸城,cq,vegnet.com.cn,2016-11-10,2018-04-24,9,...,微风,无持续风向,41,1,优,102.0,4.200000,5.844444,2.733333,4.711111
13,西兰花,7.4,7.6,7.2,重庆双福国际农贸城,cq,vegnet.com.cn,2016-11-10,2018-04-24,9,...,微风,无持续风向,41,1,优,102.0,4.133333,5.777778,2.600000,4.777778


In [201]:
data.describe()

,v_price,h_price,l_price,lWendu,hWendu,aqi,aqiLevel,cpi,h_3,h_9,l_3,l_9
count,5225.000000,5225.000000,5225.000000,5225.000000,5225.000000,5225.000000,5225.000000,5225.000000,5225.000000,5225.000000,5225.000000,5225.000000
mean,4.473579,5.051962,3.894469,12.742584,18.257033,72.896651,1.927464,101.437780,5.053289,5.051464,3.895962,3.894873
std,3.900365,4.532441,3.376327,6.174572,7.585675,36.538957,0.796661,0.778561,2.706646,1.321233,2.044334,0.985026
min,0.500000,0.600000,0.000000,2.000000,5.000000,18.000000,1.000000,99.800000,1.266667,2.288889,0.833333,1.644444
25%,2.200000,2.400000,1.800000,8.000000,12.000000,46.000000,1.000000,101.200000,3.333333,4.066667,2.533333,3.177778
50%,3.400000,4.000000,3.000000,11.000000,17.000000,67.000000,2.000000,101.600000,4.333333,4.777778,3.366667,3.788889
75%,5.500000,6.000000,5.000000,16.000000,24.000000,89.000000,2.000000,101.900000,6.000000,5.844444,4.800000,4.455556
max,38.000000,50.000000,28.000000,30.000000,40.000000,214.000000,5.000000,103.100000,20.266667,11.000000,14.800000,7.977778


以上是对原始数据集中的几个数值型变量进行的基本统计分析，可以看到均值，标准差，中位数和最值等基本描述统计量。

### 1.3 数据分割与重排
接下来，先将数据集中的蔬菜名称转换为独热编码（One Hot Encoding），然后分成训练和测试两个子集并打乱数据顺序，消除数据集中由于顺序产生的偏差，分割比例为80%数据用于训练，20%用于测试。

In [178]:
from sklearn.model_selection import train_test_split
from pandas import get_dummies

features_encoded = get_dummies(features, columns=['v_name','area','v_market'])
X_train, X_test, y_train, y_test = train_test_split(features_encoded, prices, test_size=0.2, random_state=42)

print('dataset train: {} {}, test: {} {}'.format(
    X_train.shape, y_train.shape, X_test.shape, y_test.shape))
features_encoded.head()

dataset train: (5896, 41) (5896, 2), test: (1475, 41) (1475, 2)


,source,updateTime,insertTime,yWendu,bWendu,tianqi,fengli,fengxiang,aqi,aqiLevel,...,v_name_葱头,v_name_蒜薹,v_name_西红柿,v_name_金针菇,v_name_青椒,v_name_韭菜,v_name_香菇,v_name_黄瓜,area_cd,v_market_四川成都龙泉聚和(国际)果蔬菜交易中心
9,vegnet.com.cn,2016-03-01,2018-04-19,6,19,晴,微风,南风,137,3,...,0,0,0,0,0,0,0,0,1,1
10,vegnet.com.cn,2016-03-01,2018-04-19,6,19,晴,微风,南风,137,3,...,0,0,0,0,0,0,0,0,1,1
11,vegnet.com.cn,2016-03-01,2018-04-19,6,19,晴,微风,南风,137,3,...,0,0,0,0,0,0,0,0,1,1
12,vegnet.com.cn,2016-03-01,2018-04-19,6,19,晴,微风,南风,137,3,...,0,0,0,0,0,0,0,0,1,1
13,vegnet.com.cn,2016-03-01,2018-04-19,6,19,晴,微风,南风,137,3,...,0,1,0,0,0,0,0,0,1,1


独热编码后，变量增加到了41个，其中蔬菜名称变成了一系列数值型数据。

## 2. 训练模型

### 2.1 线性回归模型

根据前面对数据进行的探索性分析，我们得到如下结论：

1. 蔬菜最高价与“最低价”、“白天温度”、“夜晚温度”、“前3天最高价均值”、“前9天最高价均值”、“前3天最低价均值”、“前9天最低价均值”具有相关关系；
2. 蔬菜最低价与“最高价”、“前3天最高价均值”、“前9天最高价均值”、“前3天最低价均值”、“前9天最低价均值”具有相关关系；

因此首先想到的是通过多元线性回归建立预测模型。首先需要在分割好的数据集中删掉不相关的变量，计算需要的变量。

In [166]:
X_train_lm = X_train.drop([
    'source', 
    'updateTime', 
    'insertTime', 
    'tianqi', 
    'fengli', 
    'fengxiang', 
    'aqi', 
    'aqiInfo', 
    'aqiLevel', 
    'cpi'
], axis=1)

X_test_lm = X_test.drop([
    'source', 
    'updateTime', 
    'insertTime', 
    'tianqi', 
    'fengli', 
    'fengxiang', 
    'aqi', 
    'aqiInfo', 
    'aqiLevel', 
    'cpi',
], axis=1)

X_train_lm.head()

,yWendu,bWendu,h_3,h_9,l_3,l_9,v_name_冬瓜,v_name_南瓜,v_name_土豆,v_name_大白菜,...,v_name_葱头,v_name_蒜薹,v_name_西红柿,v_name_金针菇,v_name_青椒,v_name_韭菜,v_name_香菇,v_name_黄瓜,area_cd,v_market_四川成都龙泉聚和(国际)果蔬菜交易中心
6241,0,12,3.666667,4.611111,2.166667,2.600000,0,0,0,0,...,1,0,0,0,0,0,0,0,1,1
2532,14,23,2.333333,2.866667,1.933333,2.400000,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
5247,14,20,4.766667,5.988889,3.600000,4.288889,0,1,0,0,...,0,0,0,0,0,0,0,0,1,1
5727,8,13,4.166667,5.355556,3.200000,4.177778,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
3299,8,13,2.100000,3.800000,1.700000,3.088889,0,0,0,0,...,0,0,0,0,0,0,1,0,1,1


下面将使用网格搜索和K折交叉验证的方法，基于线性回归模型建立对价格的预测。

In [167]:
from sklearn.metrics import r2_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

def r2_score_metric(y_true, y_pred):
    score = r2_score(y_true, y_pred)
    return score

def fit_model(X, y, regressor, params):
    cross_validator = KFold(n_splits=5, shuffle=True, random_state=42)
    scoring_func = make_scorer(r2_score_metric)
    grid = GridSearchCV(estimator=regressor, param_grid=params, scoring=scoring_func, cv=cross_validator)
    grid = grid.fit(X, y)
    return grid.best_estimator_

linear_regressor = fit_model(X_train_lm, y_train, LinearRegression(), {
        'fit_intercept': [True, False],
        'normalize': [True, False],
    })

print('paramter for the optimal model {}'.format(linear_regressor.get_params()))

paramter for the optimal model {'copy_X': True, 'normalize': True, 'n_jobs': 1, 'fit_intercept': True}


In [168]:
y_pred = linear_regressor.predict(X_test_lm)
r2 = r2_score_metric(y_test, y_pred)

print('optimal model has R^2 score {:,.2f} on test data'.format(r2))

optimal model has R^2 score 0.88 on test data


### 2.2 决策树模型

In [202]:
features_encoded = get_dummies(features, columns=['v_name', 'tianqi','area','v_market','aqiInfo'])
X_train, X_test, y_train, y_test = train_test_split(features_encoded, prices, test_size=0.2, random_state=42)

X_train = X_train.drop([
    'source', 
    'updateTime', 
    'insertTime', 
    'fengli', 
    'fengxiang', 
    'aqiLevel', 
    'cpi',
], axis=1)

X_test = X_test.drop([
    'source', 
    'updateTime', 
    'insertTime', 
    'fengli', 
    'fengxiang', 
    'aqiLevel', 
    'cpi',
], axis=1)

print('dataset train: {} {}, test: {} {}'.format(
    X_train.shape, y_train.shape, X_test.shape, y_test.shape))

dataset train: (4180, 71) (4180, 2), test: (1045, 71) (1045, 2)


In [203]:
from sklearn.tree import DecisionTreeRegressor

dt_regressor = fit_model(X_train, y_train, DecisionTreeRegressor(), {
        'max_depth': range(1, 11),
    })

print('paramter for the optimal model {}'.format(dt_regressor.get_params()))

paramter for the optimal model {'presort': False, 'splitter': 'best', 'min_impurity_decrease': 0.0, 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'criterion': 'mse', 'random_state': None, 'min_impurity_split': None, 'max_features': None, 'max_depth': 9}


In [204]:
y_pred = dt_regressor.predict(X_test)
r2 = r2_score_metric(y_test, y_pred)
print('optimal model has R^2 score {:,.2f} on test data'.format(r2))

optimal model has R^2 score 0.65 on test data


In [172]:
# print features_encoded[0:2]
# result = dt_regressor.predict(features_encoded)

# print result
# result = {"h_price_pred": result[:,0],
#           "l_price_pred": result[:,1]}
# print result
# result = pd.DataFrame(result)
# print result
# data1 = data
# #print data1
# #data1 = data1.reset_index(drop=True)
# data1.head()
# result = data.join(result)
# result.head()
# #result.to_csv("out_chengdu1.csv", sep=',', index=True, encoding='utf-8')

           source  updateTime  insertTime  yWendu  bWendu fengli fengxiang  \
9   vegnet.com.cn  2016-03-01  2018-04-19       6      19     微风        南风   
10  vegnet.com.cn  2016-03-01  2018-04-19       6      19     微风        南风   

    aqi  aqiLevel aqiInfo              ...               tianqi_阴~多云  \
9   137         3    轻度污染              ...                         0   
10  137         3    轻度污染              ...                         0   

    tianqi_阴~小雨  tianqi_阴~晴  tianqi_阴~阵雨  tianqi_阵雨  tianqi_阵雨~多云  \
9             0           0            0          0             0   
10            0           0            0          0             0   

    tianqi_阵雨~小雨  tianqi_阵雨~阴  area_cd  v_market_四川成都龙泉聚和(国际)果蔬菜交易中心  
9              0            0        1                             1  
10             0            0        1                             1  

[2 rows x 78 columns]


ValueError: could not convert string to float: 鑹�

In [205]:
result = dt_regressor.predict(features_encoded.drop([
    'source', 
    'updateTime', 
    'insertTime', 
    'fengli', 
    'fengxiang', 
    'aqiLevel', 
    'cpi',
], axis=1))
print result
result = {"h_price_pred": result[:,0],
          "l_price_pred": result[:,1]}
result = pd.DataFrame(result)
data1 = data.reset_index(drop=True)
data1.head()
result = data1.join(result)
result.head()
#result.to_csv("out_chengdu.csv", sep=',', index=True, encoding='utf-8')
result.to_csv("out_chongqing.csv", sep=',', index=True, encoding='utf-8')

[[ 3.08438406  2.33673913]
 [ 3.08438406  2.33673913]
 [ 3.08438406  2.33673913]
 ...
 [27.57142857 24.        ]
 [ 5.51428571  4.54285714]
 [ 3.08438406  2.33673913]]


## 3. 结论